In [ ]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive, auth
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

In [ ]:
import random
import torch
from torch import nn
import numpy as np
args = dict()
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# reset_seeds()

In [ ]:
from data_utils import (Pamap2Handler, FormatPamap)

SUBJECTS = list(range(1,9))

handler = Pamap2Handler(DATA_DIR)
formatter = FormatPamap()
dfs_train = [formatter.transform(handler.get_protocol_subject(i)) for i in SUBJECTS]
[len(df)//200 for df in dfs_train]

In [ ]:
git_pull()

import importlib

import PPG
import preprocessing_utils
from PPG import FullTrainer

import RegressionHR
importlib.reload(RegressionHR)

from RegressionHR import FullTrainer
from RegressionHR import PceLstmDefaults
from RegressionHR import PceLstmModel
from RegressionHR import TrainerJoint
from RegressionHR import  UtilitiesData
from RegressionHR import *
from RegressionHR import Preprocessing
import Models

importlib.reload(Models.BaseModels)

# importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
# importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
# importlib.reload(PPG.NoHrPceLstmModel)
# importlib.reload(PPG.TrainerXY)
# importlib.reload(PPG.TrainerIS)
# importlib.reload(PPG.FullTrainer)
# importlib.reload(PceLstmDefaults)
# importlib.reload(preprocessing_utils)

importlib.reload(RegressionHR.FullTrainer)
importlib.reload(RegressionHR.PceLstmDefaults)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(preprocessing_utils)
importlib.reload(RegressionHR.TrainerJoint)
importlib.reload(RegressionHR.UtilitiesData)
importlib.reload(RegressionHR.PceLstmModel)
importlib.reload(preprocessing_utils)
# import imp
# for module in sys.modules.values():
#     importlib.reload(module)



In [ ]:
import random 

options = {
  'ts_sub': [5],
  'batch_size': [64],
  'weight_decay': [0.0001, 0.00001, 0.0],
  'lr': [0.001, 0.005, 0.0001, 0.0005],
  'ts_per_sample':[30, 40, 50],
  "layer_sizes": [(4,4,4), (8,8,8), (16, 16, 16), (32,32,32), (64,64,64)],
  "skip_mapping": [((0,2),(0,3),(1,3)), ((0,2),(0,3)), ((1,3),(0,3)),  ((0,3),)],
  "dropout_rate": [0.1]
  }


from  Optimization.Optimizers import RandomSearch
import json
import pickle

filename = "pamap_ffnn_discriminator_results.pkl"
save_path = os.path.join(STORE_DIR, filename)

try:
  with open(save_path, "rb") as f:
    results = pickle.load(f)
except FileNotFoundError:
  results = list()

full_trainer = RegressionHR.FullTrainer.IteractiveFFNNFullTrainerJointValidation(dfs_train, args["device"], nepoch =  500)

searcher = RandomSearch(full_trainer, options)
searcher.results = results
while True:
  searcher.fit(1)
  with open(save_path, "wb") as f:
    pickle.dump(searcher.results, f)